## Install the geocoder

In [ ]:
!pip install geocoder


## Get the survey data

- [survey](https://docs.google.com/forms/d/1oF7us4ASuFpF6iOhn4g5MnXf4Z8E5EPaIe2mMdPw_cM/edit)
- [data](https://docs.google.com/spreadsheets/d/1qPtvXZJ-IoNXdbVcPxr4buVLUONGsYS7-xNGqPFP144/edit#gid=696092516)


In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vSg8WVXVb47Tri6-JLpYZ-rxXUr3iX5HRD23rzK5XmABGpQI3jfSZk9btVTiguQZIgIWAmecSjbXvUM/pub?gid=696092516&single=true&output=csv')
                 

In [ ]:
df

## Rename columns

In [ ]:
df.columns=['timestamp','zip','work','email']

In [ ]:
df

## Geocode the zipcodes

In [ ]:
import geocoder

In [ ]:
df['lat'] = ''
df['lon'] = ''

In [ ]:
for index, row in df.iterrows():
    g = geocoder.mapbox(row.zip,key='pk.eyJ1IjoiY2h3b2lwcm9qZWN0IiwiYSI6ImNrdTRjZXkwaTRwaWwycXBtY290MjA5d3gifQ.iyENw-MBBY3_9tMPv7frFg')
    print(g.latlng)
    df.loc[index,'lat'] = g.latlng[0]
    df.loc[index,'lon'] = g.latlng[1]
    

In [ ]:
df

### Get average lat/lon's

In [ ]:
# average latitude
latitude = df.lat.mean()
latitude

In [ ]:
# average longitude
longitude = df.lon.mean()
longitude

## Map it!

In [ ]:
import folium

In [ ]:
from folium.plugins import FloatImage

# image_file = 'image.PNG'

# FloatImage(image_file, bottom=0, left=86).add_to(mymap)

In [ ]:
# map setup
m = folium.Map(location = [34.0703726,-118.4453799], tiles="Stamen Terrain", zoom_start=10)

# add radius circles around ucla
radii = [40,30,20,10]
for radius in radii:
    folium.Circle(
        popup=str(radius)+" km radius from UCLA",
        tooltip=str(radius)+" km radius from UCLA",
        radius=radius*1000,
        location=[34.0703726,-118.4453799],
        color="crimson",
        fill=True,
        opacity=0.3,
        fill_opacity=0.05
    ).add_to(m)

# add markers with tooltips colored by desired work mode
for index, row in df.iterrows():
    tooltip = 'I live in zipcode '+str(row.zip)+' and want to work '+row.work
    # color
    if row.work == '100% remote':
        color = 'red'
    elif row.work == 'hybrid':
        color = 'darkblue'
    else:
        color = 'green'
    folium.Marker([row.lat, row.lon], 
                  popup=row.work, 
                  tooltip=tooltip,
                  icon=folium.Icon(color=color,icon='user'),
                 ).add_to(m)
    
FloatImage('https://raw.githubusercontent.com/yohman/interactive-engagement/main/legend3.png',bottom=3,left=2).add_to(m)
# output the map
m

In [ ]:
# save the interactive maps as an html file
m.save('index.html')